In [ ]:
#network.add(tf.keras.layers.Conv2D(16, (3, 3), (1, 1), 'same', activation='relu', input_shape=(128, 128, 1)))
# 16은 사용할 커널필터의 개수 (3,3)은 커널의 크기
# (1,1)은 x, y축으로 1칸씩 이동, same은 padding 알고리즘
# 실행하고 나면 (128, 128, 16)이 된다.
# 합성곱 : 필터를 통해 이미지의 특징을 강조한다.
# 풀링 : 이미지의 차원을 요약한다.
# 평균풀링 : 각 필터의 평균값을 계산한다. 즉 (1,1)부터 (128, 128)의 16개 커널에 대해서 각각 16개의 평균값을 계산한다.
#GlobalAveragePooling2D vs Flatten()
#합성곱 layer와 pooling은 일반적인 특성, 밀집층은 우리의 특별한 목적에 관해 학습을 제공한다.

In [ ]:
import tensorflow as tf
import seaborn as sns
import zipfile
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/ColabDrive/Computer_Vision_Masterclass/Datasets/homer_bart_2.zip"
zip_object = zipfile.ZipFile(file=path, mode='r')
zip_object.extractall('./')
zip_object.close()

In [ ]:
train_generator = ImageDataGenerator(rescale=1./255,
                                     rotation_range = 7,
                                     horizontal_flip = True,
                                     zoom_range=0.2)
train_dataset = train_generator.flow_from_directory('/content/homer_bart_2/training_set',
                                                    target_size = (256, 256),
                                                    batch_size = 8,
                                                    class_mode = 'categorical',
                                                    shuffle = True
                                                    )

Found 215 images belonging to 2 classes.


In [ ]:
train_dataset.class_indices

{'bart': 0, 'homer': 1}

In [ ]:
test_generator = ImageDataGenerator(rescale=1./255)
test_dataset = test_generator.flow_from_directory('/content/homer_bart_2/test_set',
                                                  target_size = (256, 256),
                                                  batch_size = 1,
                                                  class_mode = 'categorical',
                                                  shuffle = True
                                                  )

Found 54 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top = False,
                                            input_tensor = Input(shape=(256,256,3)))

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
print(len(base_model.layers)) #layer가 trainable한지
for layer in base_model.layers:
  layer.trainable = False # 전이학습을 위해 사전 모델의 가중치 업데이트를 막음
base_model.summary()

In [ ]:
print(base_model.output) #마지막 출력층의 특성 파악
head_model = base_model.output # head_model에 연결 -> 평탄화 작업 필요
head_model = GlobalAveragePooling2D()(head_model) #Sequential 클래스 대신 Model 클래스를 사용하기 때문에 모델과 모델을 연결하는 역할
head_model = Dense(units=1025, activation='relu')(head_model) #why (평탄화 된 뉴런의 수 + 출력층의 뉴런 수) // 2
head_model = Dropout(rate = 0.2)(head_model)
head_model = Dense(units=1025, activation='relu')(head_model)
head_model = Dropout(rate = 0.2)(head_model) #random?
head_model = Dense(2, activation='softmax')(head_model)

In [ ]:
network = Model(inputs = base_model.input, outputs = head_model) #2개의 모델을 연결
network.summary()

In [ ]:
network.compile(optimizer='Adam', loss = 'categorical_crossentropy', metrics=['accuracy'])
history = network.fit(train_dataset, epochs = 50)

In [ ]:
from sklearn.metrics import accuracy_score
prediction = network.predict(test_dataset)
prediction = np.argmax(prediction, axis=1)
accuracy_score(test_dataset.classes, prediction)

54/54 [==============================] - 15s 258ms/step


0.5555555555555556

In [ ]:
#Fine Tuning : 기존의 합성곱 신경망 또한 같이 가중치를 조정하는 것 -> 학습률을 작게 지정하여 너무 크게 변하지 않도록, 오버피팅 되지 않도록 조정함
# 되도록 마지막 layer에 대해서 fine tuning을 한다. input layer에 가까워질 수록 일반적인 특성에 관한 것이기 때문이다.

In [ ]:
base_model.trainable = True
fine_tuning_at = 140
for layer in base_model.layers[:fine_tuning_at]:
  layer.trainable = False

In [ ]:
network.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = network.fit(train_dataset, epochs = 50)

In [ ]:
from sklearn.metrics import accuracy_score
prediction = network.predict(test_dataset)
prediction = np.argmax(prediction, axis=1)
accuracy_score(test_dataset.classes, prediction)

In [ ]:
######### 숙제 ##########

In [ ]:
path = "/content/drive/MyDrive/ColabDrive/Computer_Vision_Masterclass/Datasets/cat_dog_2.zip"
zip_object = zipfile.ZipFile(file=path, mode='r')
zip_object.extractall('./')
zip_object.close()

In [ ]:
train_generator = ImageDataGenerator(rescale=1./255,)
train_dataset = train_generator.flow_from_directory('/content/cat_dog_2/training_set',
                                                    target_size = (128, 128),
                                                    batch_size = 128,
                                                    class_mode = 'binary',
                                                    shuffle = True
                                                    )
test_generator = ImageDataGenerator(rescale=1./255)
test_dataset = test_generator.flow_from_directory('/content/cat_dog_2/test_set',
                                                  target_size = (128, 128),
                                                  batch_size = 1,
                                                  class_mode = 'binary',
                                                  shuffle = False
                                                  )

Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128,3))
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
head_model = base_model.output
head_model = tf.keras.layers.GlobalAveragePooling2D()(head_model)
head_model = Dense(641, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(641, activation='relu')(head_model)
head_model = Dropout(0.2)(head_model)
head_model = Dense(1, activation='sigmoid')(head_model)

In [ ]:
network = Model(inputs=base_model.input, outputs = head_model)

In [ ]:
network.compile(loss='binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
history = network.fit(train_dataset, epochs=5)

Epoch 1/5
32/32 [==============================] - 88s 3s/step - loss: 0.1767 - accuracy: 0.9302
Epoch 2/5
32/32 [==============================] - 64s 2s/step - loss: 0.0717 - accuracy: 0.9732
Epoch 3/5
32/32 [==============================] - 58s 2s/step - loss: 0.0508 - accuracy: 0.9822
Epoch 4/5
32/32 [==============================] - 57s 2s/step - loss: 0.0278 - accuracy: 0.9912
Epoch 5/5
32/32 [==============================] - 64s 2s/step - loss: 0.0146 - accuracy: 0.9962


In [ ]:
network.evaluate(test_dataset)

1000/1000 [==============================] - 27s 26ms/step - loss: 0.1064 - accuracy: 0.9660


[0.106431744992733, 0.9660000205039978]

In [ ]:
#############fine_tuning###############3
fine_tuning_at = 100
base_model.trainable = True
for layer in base_model.layers[:fine_tuning_at]:
  layer.trainable=False

In [ ]:
network.compile(loss = 'binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
history = network.fit(train_dataset, epochs=5)

32/32 [==============================] - 124s 4s/step - loss: 0.2550 - accuracy: 0.9260


In [ ]:
prediction = network.predict(test_dataset)

1000/1000 [==============================] - 29s 29ms/step


In [ ]:
prediction = np.argmax(prediction, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(test_dataset.classes, prediction)

0.5

In [ ]:
network.evaluate(test_dataset)

1000/1000 [==============================] - 26s 24ms/step - loss: 4.2580 - accuracy: 0.6590


[4.257970809936523, 0.6589999794960022]